**Goal**: this notebook is iseful to prepare data for DEG Analysis.

We want to build four different datasets:

1. RNA-seq data (PR/CR)
2. RNA-seq data (SD/PD)
3. Normalized RNA-seq data (PR/CR)
4. Normalized RNA-seq data (SD/PD)

The further goal of DEG analysis is to identify the genes that are differentially expressed with respect to the response, in particular this information is included in the variable **Harmonized_Confirmed_BOR** that contains four different levels:

1. **CR**: Complete Response
2. **PR**: Partial Response
3. **SD**: Stable Disease
4. **PD**: Progressive Disease

There is also another level, **NE** (Not Evaluable) that will not be considered for the analysis.

### Import Utils and Setup

In [ ]:
## Communication drive-colab
from google.colab import drive
import warnings
import os

## Data Structure and Data Analysis
import pandas as pd
import numpy as np

In [ ]:
## Mount drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [ ]:
## Toggle as needed
warnings.filterwarnings("ignore")

## To see the maximum number of columns
pd.set_option("display.max_columns", None)

## Source path
source_path_data = "/content/drive/MyDrive/Tesi/Code/Personal_Code/Saved/Data/"

## Save path
save_path_data   = "/content/drive/MyDrive/Tesi/Code/Personal_Code/Saved/Data/"

### Read Clinical Data (Shape: n = 152, p = 38)

We need to work on the Clinical data, in order to extract the patients ID with the four different conditions, and then filter the RNA-seq data.

In [ ]:
## Load Data
master_annotations_df_hq = pd.read_csv(source_path_data + "master_annotations_df_hq.csv", sep = ",", usecols = lambda x: x != "Unnamed: 0")

In [ ]:
## Check
print("Shape is: ", master_annotations_df_hq.shape)
display(master_annotations_df_hq.head())

Shape is:  (152, 38)


,WES_Cohort_1,WES_Cohort_2,WES_All,RNA_Cohort_1,RNA_Cohort_2,RNA_All,Institution,Harmonized_SU2C_Participant_ID_v2,Harmonized_SU2C_WES_Tumor_Sample_ID_v2,Harmonized_SU2C_WES_Normal_Sample_ID_v2,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Pre-treatment_RNA_Sample_QC,Patient_Age_at_Diagnosis,Patient_Sex,Patient_Race,Patient_Smoking_Status,Patient_Smoking_Pack_Years_Harmonized,Histology_Harmonized,Histology_Detail,Initial_Stage,Initial_Stage_Substage,PDL1_TPS,PDL1_TPS_Description,Local_Antibody_Clone,Clinical_Driver,Sequencing_Platform,Advanced_Diagnosis_Date,Line_of_Therapy,Agent_PD1,Agent_PD1_Category,Prior_Platinum,Prior_TKI,Harmonized_PFS_Event,Harmonized_PFS_Days,Harmonized_Confirmed_BOR,Harmonized_BOR_RECIST,Harmonized_OS_Event,Harmonized_OS_Days
0,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO10,NaN,NaN,SU2CLC-CLE-NIVO10-T1,Flag,55,F,2.0,1.0,30.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-321.0,3.0,Nivolumab,PD(L)1,1.0,0.0,1.0,63.0,PD,NaN,1.0,86.0
1,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO18,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO18-N1,SU2CLC-CLE-NIVO18-T1,Keep,68,F,0.0,0.0,0.0,Adeno,NaN,2.0,A,NaN,NaN,NaN,EGFR,NaN,-533.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,50.0,PD,NaN,1.0,161.0
2,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO19,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO19-N1,SU2CLC-CLE-NIVO19-T1,Keep,57,F,2.0,2.0,15.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-35.0,1.0,Nivolumab,PD(L)1,0.0,0.0,1.0,297.0,PR,NaN,1.0,297.0
3,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO2,NaN,NaN,SU2CLC-CLE-NIVO2-T1,Keep,63,F,0.0,0.0,0.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-262.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,68.0,PD,NaN,1.0,123.0
4,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO20,NaN,NaN,SU2CLC-CLE-NIVO20-T1,Keep,63,F,0.0,1.0,35.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,KRAS,NaN,-301.0,2.0,Nivolumab,PD(L)1,1.0,0.0,1.0,273.0,PR,NaN,1.0,273.0


Considering two features: **Harmonized_SU2C_RNA_Tumor_Sample_ID_v2** and **Harmonized_Confirmed_BOR**, extract 2 different subset:

1. Harmonized_SU2C_RNA_Tumor_Sample_ID_v2 & Harmonized_Confirmed_BOR == PR/CR
2. Harmonized_SU2C_RNA_Tumor_Sample_ID_v2 & Harmonized_Confirmed_BOR == SD/PD

**PR/CR**: Response

**SD/PD**: Resistance

In [ ]:
## Count levels of BOR and chech if there are NaN
print("Total observations number with NaN: ", master_annotations_df_hq["Harmonized_Confirmed_BOR"].isna().sum())
print("\n")

display(master_annotations_df_hq.groupby("Harmonized_Confirmed_BOR").size())

Total observations number with NaN:  13




Harmonized_Confirmed_BOR
CR     8
NE     3
PD    45
PR    44
SD    39
dtype: int64

In [ ]:
## Create a function that takes in input the dataframe and make a subset
## This function contains two internal variables thar are fixed for the extractino of subset
## Harmonized_SU2C_RNA_Tumor_Sample_ID_v2 and Harmonized_Confirmed_BOR

def make_subset(data, feature, value_1, value_2):
    """Input: data    ---> dataset
              feature ---> variable on which we want to filter
              value   ---> value that we want to filter"""
    """Output: subset"""
    rna_patients = data[(data[feature] == value_1) | (data[feature] == value_2)]
    ## Check
    print("Shape is: ", rna_patients.shape)
    display(rna_patients.head())
    print("\n")

    ## Extract only the ID patient and the BOR
    rna_patients = rna_patients.loc[:, ["Harmonized_SU2C_RNA_Tumor_Sample_ID_v2", "Harmonized_Confirmed_BOR"]]
    ## Check
    print("Shape is: ", rna_patients.shape)
    display(rna_patients.head())

    return rna_patients

In [ ]:
## Extract the first one (PR/CR)
rna_patients_pr_cr = make_subset(master_annotations_df_hq, "Harmonized_Confirmed_BOR", "PR", "CR")
## Class: Response

Shape is:  (52, 38)


,WES_Cohort_1,WES_Cohort_2,WES_All,RNA_Cohort_1,RNA_Cohort_2,RNA_All,Institution,Harmonized_SU2C_Participant_ID_v2,Harmonized_SU2C_WES_Tumor_Sample_ID_v2,Harmonized_SU2C_WES_Normal_Sample_ID_v2,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Pre-treatment_RNA_Sample_QC,Patient_Age_at_Diagnosis,Patient_Sex,Patient_Race,Patient_Smoking_Status,Patient_Smoking_Pack_Years_Harmonized,Histology_Harmonized,Histology_Detail,Initial_Stage,Initial_Stage_Substage,PDL1_TPS,PDL1_TPS_Description,Local_Antibody_Clone,Clinical_Driver,Sequencing_Platform,Advanced_Diagnosis_Date,Line_of_Therapy,Agent_PD1,Agent_PD1_Category,Prior_Platinum,Prior_TKI,Harmonized_PFS_Event,Harmonized_PFS_Days,Harmonized_Confirmed_BOR,Harmonized_BOR_RECIST,Harmonized_OS_Event,Harmonized_OS_Days
2,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO19,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO19-N1,SU2CLC-CLE-NIVO19-T1,Keep,57,F,2.0,2.0,15.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-35.0,1.0,Nivolumab,PD(L)1,0.0,0.0,1.0,297.0,PR,NaN,1.0,297.0
4,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO20,NaN,NaN,SU2CLC-CLE-NIVO20-T1,Keep,63,F,0.0,1.0,35.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,KRAS,NaN,-301.0,2.0,Nivolumab,PD(L)1,1.0,0.0,1.0,273.0,PR,NaN,1.0,273.0
7,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO3,SU2CLC-CLE-NIVO3-T1,SU2CLC-CLE-NIVO3-N1,SU2CLC-CLE-NIVO3-T1,Keep,57,F,0.0,0.0,0.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,ALK,NaN,-497.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,392.0,PR,NaN,1.0,636.0
10,NaN,1.0,1.0,NaN,1.0,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO5,SU2CLC-CLE-NIVO5-T1,SU2CLC-CLE-NIVO5-N1,SU2CLC-CLE-NIVO5-T1,Keep,61,M,0.0,1.0,60.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-686.0,2.0,Nivolumab,PD(L)1,1.0,0.0,1.0,169.0,PR,NaN,1.0,547.0
11,NaN,NaN,NaN,NaN,1.0,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO52,NaN,NaN,SU2CLC-CLE-NIVO52-T1,Flag,64,M,0.0,1.0,75.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,EGFR,NaN,-1054.0,6.0,Nivolumab,PD(L)1,1.0,1.0,1.0,622.0,PR,NaN,1.0,911.0




Shape is:  (52, 2)


,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Harmonized_Confirmed_BOR
2,SU2CLC-CLE-NIVO19-T1,PR
4,SU2CLC-CLE-NIVO20-T1,PR
7,SU2CLC-CLE-NIVO3-T1,PR
10,SU2CLC-CLE-NIVO5-T1,PR
11,SU2CLC-CLE-NIVO52-T1,PR


In [ ]:
## Extract the second one (PR/CR)
rna_patients_sd_pd = make_subset(master_annotations_df_hq, "Harmonized_Confirmed_BOR", "SD", "PD")
## Class: Resistance

Shape is:  (84, 38)


,WES_Cohort_1,WES_Cohort_2,WES_All,RNA_Cohort_1,RNA_Cohort_2,RNA_All,Institution,Harmonized_SU2C_Participant_ID_v2,Harmonized_SU2C_WES_Tumor_Sample_ID_v2,Harmonized_SU2C_WES_Normal_Sample_ID_v2,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Pre-treatment_RNA_Sample_QC,Patient_Age_at_Diagnosis,Patient_Sex,Patient_Race,Patient_Smoking_Status,Patient_Smoking_Pack_Years_Harmonized,Histology_Harmonized,Histology_Detail,Initial_Stage,Initial_Stage_Substage,PDL1_TPS,PDL1_TPS_Description,Local_Antibody_Clone,Clinical_Driver,Sequencing_Platform,Advanced_Diagnosis_Date,Line_of_Therapy,Agent_PD1,Agent_PD1_Category,Prior_Platinum,Prior_TKI,Harmonized_PFS_Event,Harmonized_PFS_Days,Harmonized_Confirmed_BOR,Harmonized_BOR_RECIST,Harmonized_OS_Event,Harmonized_OS_Days
0,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO10,NaN,NaN,SU2CLC-CLE-NIVO10-T1,Flag,55,F,2.0,1.0,30.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-321.0,3.0,Nivolumab,PD(L)1,1.0,0.0,1.0,63.0,PD,NaN,1.0,86.0
1,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO18,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO18-N1,SU2CLC-CLE-NIVO18-T1,Keep,68,F,0.0,0.0,0.0,Adeno,NaN,2.0,A,NaN,NaN,NaN,EGFR,NaN,-533.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,50.0,PD,NaN,1.0,161.0
3,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO2,NaN,NaN,SU2CLC-CLE-NIVO2-T1,Keep,63,F,0.0,0.0,0.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-262.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,68.0,PD,NaN,1.0,123.0
6,NaN,NaN,NaN,NaN,1.0,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO24,NaN,NaN,SU2CLC-CLE-NIVO24-T1,Flag,73,F,0.0,1.0,82.0,Squamous,NaN,3.0,A,NaN,NaN,NaN,NaN,NaN,-11.0,2.0,Nivolumab,PD(L)1,1.0,0.0,1.0,172.0,SD,NaN,1.0,172.0
8,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO31,NaN,NaN,SU2CLC-CLE-NIVO31-T1,Flag,83,M,0.0,1.0,4.0,Squamous,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,-4.0,1.0,Nivolumab,PD(L)1,0.0,0.0,1.0,12.0,PD,NaN,1.0,98.0




Shape is:  (84, 2)


,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Harmonized_Confirmed_BOR
0,SU2CLC-CLE-NIVO10-T1,PD
1,SU2CLC-CLE-NIVO18-T1,PD
3,SU2CLC-CLE-NIVO2-T1,PD
6,SU2CLC-CLE-NIVO24-T1,SD
8,SU2CLC-CLE-NIVO31-T1,PD


In [ ]:
## Check
print("Total observations number: ", master_annotations_df_hq.shape[0])
print("Total observations number where BOR is NE: ", master_annotations_df_hq[(master_annotations_df_hq["Harmonized_Confirmed_BOR"] == "NE")].shape[0])
print("Total observations number with NaN: ", master_annotations_df_hq["Harmonized_Confirmed_BOR"].isna().sum())
print("Total observations number where BOR is PR, CR, SD, PD: ", rna_patients_pr_cr.shape[0] + rna_patients_sd_pd.shape[0])

Total observations number:  152
Total observations number where BOR is NE:  3
Total observations number with NaN:  13
Total observations number where BOR is PR, CR, SD, PD:  136


In [ ]:
## Consider only the samples where BOR is PR, CR, SD, PD
expression_BOR = ["PR", "CR", "SD", "PD"]

# Create subset excluding rows with values in exclude_values list in column 'B'
master_annotations_df_hq_clean = master_annotations_df_hq[master_annotations_df_hq["Harmonized_Confirmed_BOR"].isin(expression_BOR)]
## The result must be 136

## Check
print("Shape is: ", master_annotations_df_hq_clean.shape)

Shape is:  (136, 38)


Now, it's possible modify the clinical table in the following way:

- if Harmonized_Confirmed_BOR == **PR/CR** , **Response**
- if Harmonized_Confirmed_BOR == **SD/PD** , **Resistance**

In [ ]:
## Modify Harmonized_Confirmed_BOR

## Define the mapping dictionary
substitutions = {"PR": "Response","CR":"Response", "SD":"Resistance", "PD":"Resistance"}
## Substitute
master_annotations_df_hq_clean["BOR_cat"] = [substitutions[x] if x in substitutions else x for x in master_annotations_df_hq_clean["Harmonized_Confirmed_BOR"]]

In [ ]:
## Check
print("Shape is: ", master_annotations_df_hq_clean.shape)
display(master_annotations_df_hq_clean.head())

## The new variable must be:
## Response   (n = 52)
## Resistance (n = 84)

Shape is:  (136, 39)


,WES_Cohort_1,WES_Cohort_2,WES_All,RNA_Cohort_1,RNA_Cohort_2,RNA_All,Institution,Harmonized_SU2C_Participant_ID_v2,Harmonized_SU2C_WES_Tumor_Sample_ID_v2,Harmonized_SU2C_WES_Normal_Sample_ID_v2,Harmonized_SU2C_RNA_Tumor_Sample_ID_v2,Pre-treatment_RNA_Sample_QC,Patient_Age_at_Diagnosis,Patient_Sex,Patient_Race,Patient_Smoking_Status,Patient_Smoking_Pack_Years_Harmonized,Histology_Harmonized,Histology_Detail,Initial_Stage,Initial_Stage_Substage,PDL1_TPS,PDL1_TPS_Description,Local_Antibody_Clone,Clinical_Driver,Sequencing_Platform,Advanced_Diagnosis_Date,Line_of_Therapy,Agent_PD1,Agent_PD1_Category,Prior_Platinum,Prior_TKI,Harmonized_PFS_Event,Harmonized_PFS_Days,Harmonized_Confirmed_BOR,Harmonized_BOR_RECIST,Harmonized_OS_Event,Harmonized_OS_Days,BOR_cat
0,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO10,NaN,NaN,SU2CLC-CLE-NIVO10-T1,Flag,55,F,2.0,1.0,30.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-321.0,3.0,Nivolumab,PD(L)1,1.0,0.0,1.0,63.0,PD,NaN,1.0,86.0,Resistance
1,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO18,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO18-N1,SU2CLC-CLE-NIVO18-T1,Keep,68,F,0.0,0.0,0.0,Adeno,NaN,2.0,A,NaN,NaN,NaN,EGFR,NaN,-533.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,50.0,PD,NaN,1.0,161.0,Resistance
2,1.0,NaN,1.0,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO19,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO19-N1,SU2CLC-CLE-NIVO19-T1,Keep,57,F,2.0,2.0,15.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-35.0,1.0,Nivolumab,PD(L)1,0.0,0.0,1.0,297.0,PR,NaN,1.0,297.0,Response
3,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO2,NaN,NaN,SU2CLC-CLE-NIVO2-T1,Keep,63,F,0.0,0.0,0.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,-262.0,4.0,Nivolumab,PD(L)1,1.0,1.0,1.0,68.0,PD,NaN,1.0,123.0,Resistance
4,NaN,NaN,NaN,1.0,NaN,1.0,Cleveland Clinic,SU2CLC-CLE-NIVO20,NaN,NaN,SU2CLC-CLE-NIVO20-T1,Keep,63,F,0.0,1.0,35.0,Adeno,NaN,4.0,NaN,NaN,NaN,NaN,KRAS,NaN,-301.0,2.0,Nivolumab,PD(L)1,1.0,0.0,1.0,273.0,PR,NaN,1.0,273.0,Response


In [ ]:
## Check
master_annotations_df_hq_clean.groupby("BOR_cat").size()

BOR_cat
Resistance    84
Response      52
dtype: int64

In [ ]:
## Extract the patients ID
patients_id = master_annotations_df_hq_clean["Harmonized_SU2C_RNA_Tumor_Sample_ID_v2"]
## Check
print("Total Number with highest quality samples ", len(patients_id))

Total Number with highest quality samples  136


### Prepare RNA-seq Data

In [ ]:
# ## Load Annotation
# rna_counts_hq = pd.read_csv(source_path_data + "rna_counts.csv").set_index("Name")
# ## Save description genes
# description_genes = rna_counts_hq["Description"]

# ## Check
# print("Shape is: ", rna_counts_hq.shape)
# display(rna_counts_hq.head())

Now, using patients_id we can select only the patients for which we are interessed


In [ ]:
## Load CPM Data (Normalized)
rna_counts_hq_cpm = pd.read_csv(source_path_data + "cpm.csv")
## Rename column
rna_counts_hq_cpm = rna_counts_hq_cpm.rename(columns = ({"Unnamed: 0": "Name"}))
## Set index
rna_counts_hq_cpm = rna_counts_hq_cpm.set_index("Name")

## Check
print("Shape is: ", rna_counts_hq_cpm.shape)
display(rna_counts_hq_cpm.head())

Shape is:  (15924, 136)


,SU2CLC-CLE-NIVO10-T1,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO2-T1,SU2CLC-CLE-NIVO20-T1,SU2CLC-CLE-NIVO24-T1,SU2CLC-CLE-NIVO3-T1,SU2CLC-CLE-NIVO31-T1,SU2CLC-CLE-NIVO5-T1,SU2CLC-CLE-NIVO52-T1,SU2CLC-CLE-NIVO9-T1,SU2CLC-COL-1001-T1,SU2CLC-COL-1005-T1,SU2CLC-COL-1007-T1,SU2CLC-COL-1008-T1,SU2CLC-COL-1010-T1,SU2CLC-COL-1016-T1,SU2CLC-COL-1017-T1,SU2CLC-COL-1018-T1,SU2CLC-COL-1020-T1,SU2CLC-COL-1021-T1,SU2CLC-COL-1022-T1,SU2CLC-COL-1023-T1,SU2CLC-COL-1025-T1,SU2CLC-COL-1026-T1,SU2CLC-COL-1027-T1,SU2CLC-COL-1029-T1,SU2CLC-COL-1031-T1,SU2CLC-COL-1032-T1,SU2CLC-COL-1033-T1,SU2CLC-COL-1034-T1,SU2CLC-COL-1035-T1,SU2CLC-COL-1036-T1,SU2CLC-COL-1037-T1,SU2CLC-COL-1038-T1,SU2CLC-COL-1039-T2,SU2CLC-COL-1041-T1,SU2CLC-COL-1043-T2,SU2CLC-COL-1044-T1,SU2CLC-DFC-1001-T1,SU2CLC-DFC-1002-T1,SU2CLC-DFC-1003-T1,SU2CLC-DFC-1004-T1,SU2CLC-DFC-1007-T1,SU2CLC-DFC-1012-T1,SU2CLC-DFC-1013-T1,SU2CLC-DFC-1015-T2,SU2CLC-DFC-1016-T1,SU2CLC-DFC-1017-T2,SU2CLC-DFC-1018-T1,SU2CLC-DFC-1019-T1,SU2CLC-DFC-1020-T1,SU2CLC-DFC-1534-T1,SU2CLC-DFC-1535-T1,SU2CLC-DFC-1536-T1,SU2CLC-DFC-1537-T1,SU2CLC-DFC-1538-T1,SU2CLC-DFC-1539-T1,SU2CLC-DFC-DF0032-T1,SU2CLC-DFC-DF0033-T1,SU2CLC-DFC-DF0047-T1,SU2CLC-DFC-DF0107-T1,SU2CLC-DFC-DF0108-T1,SU2CLC-DFC-DF0109-T1,SU2CLC-DFC-DF0112-T1,SU2CLC-DFC-DF0241-T1,SU2CLC-DFC-DF0499-T1,SU2CLC-DFC-DF0510-T1,SU2CLC-DFC-DF0512-T1,SU2CLC-DFC-DF0561-T1,SU2CLC-DFC-DF0668-T1,SU2CLC-DFC-DF0790-T1,SU2CLC-DFC-DF0840-T1,SU2CLC-MDA-1441-T1,SU2CLC-MDA-1442-T1,SU2CLC-MDA-1443-T1,SU2CLC-MDA-1444-T1,SU2CLC-MDA-1561-T1,SU2CLC-MDA-1562-T1,SU2CLC-MDA-1563-T1,SU2CLC-MDA-1564-T1,SU2CLC-MDA-1627-T1,SU2CLC-MDA-1628-T1,SU2CLC-MDA-1629-T1,SU2CLC-MDA-1630-T1,SU2CLC-MDA-1631-T1,SU2CLC-MGH-1044-T1,SU2CLC-MGH-1054-T2,SU2CLC-MGH-1055-T1,SU2CLC-MGH-1135-T2,SU2CLC-MGH-1148-T1,SU2CLC-MGH-1149-T1,SU2CLC-MGH-1150-T1,SU2CLC-MGH-1161-T2,SU2CLC-MGH-1163-T1,SU2CLC-MGH-1169-T1,SU2CLC-MGH-1387-T1,SU2CLC-MGH-1389-T1,SU2CLC-MGH-1409-T1,SU2CLC-MGH-1411-T1,SU2CLC-MGH-1412-T1,SU2CLC-MGH-1413-T1,SU2CLC-MGH-1414-T1,SU2CLC-MGH-1415-T1,SU2CLC-MGH-1416-T1,SU2CLC-MGH-1417-T1,SU2CLC-MGH-1418-T1,SU2CLC-MGH-1487-T1,SU2CLC-MGH-1488-T1,SU2CLC-MGH-1489-T1,SU2CLC-MGH-1490-T1,SU2CLC-MGH-1492-T1,SU2CLC-MGH-1493-T1,SU2CLC-MGH-1495-T1,SU2CLC-MGH-1498-T1,SU2CLC-MGH-1500-T1,SU2CLC-MGH-1501-T1,SU2CLC-MGH-1503-T1,SU2CLC-MGH-1565-T1,SU2CLC-MGH-1568-T1,SU2CLC-MGH-1572-T1,SU2CLC-MGH-1574-T1,SU2CLC-MGH-1575-T1,SU2CLC-MSK-1364-T1,SU2CLC-MSK-1365-T1,SU2CLC-MSK-A2009-T1,SU2CLC-MSK-A2013-T1,SU2CLC-MSK-A2014-T1,SU2CLC-MSK-A2060-T1,SU2CLC-MSK-A2075-T1,SU2CLC-UCD-1124-T1,SU2CLC-UCD-1137-T1,SU2CLC-UCD-1142-T1,SU2CLC-UCD-1143-T1,SU2CLC-UCD-1557-T1,SU2CLC-UCD-1560-T1
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634.11,4.431076,0.453236,1.155171,1.009496,1.767499,3.307169,0.360152,0.850265,5.292916,52.776752,2.684743,4.189089,7.803944,0.928295,0.553878,1.350860,1.682919,0.306656,0.000000,7.350413,2.353301,4.706567,3.533557,0.432397,0.860571,4.574732,1.306453,0.495657,4.650064,17.301699,0.335699,7.226855,1.783137,1.114443,1.586550,0.926229,1.926702,3.643740,0.879785,7.798641,2.771499,17.313346,2.388004,5.376212,6.243532,3.387685,4.052297,0.651425,2.532179,32.106544,2.183914,1.458088,4.709117,6.184516,1.502689,5.528765,3.639040,3.898140,7.210490,3.907967,2.027742,2.882646,6.936726,5.329858,4.736300,0.930876,2.706959,0.427445,2.640967,0.092825,1.192969,0.480931,4.105426,8.984268,0.881750,4.555242,0.811472,5.054237,1.412726,2.761905,0.000000,6.881449,7.022262,2.075339,18.495830,0.557860,8.206746,8.156931,0.000000,5.334674,0.754773,6.509234,3.244957,1.480543,15.500587,1.390940,9.337178,3.920356,3.102597,0.994180,0.520856,5.577870,1.885142,0.966936,2.321603,3.670364,3.960003,0.027218,5.111210,0.543567,3.723049,1.090681,0.880358,7.600478,1.922739,0.875893,0.148757,1.208175,0.252068,2.935332,13.906862,4.539829,1.619512,13.348662,2.940581,3.946299,7.682567,1.270299,4.439976,2.985712,5.150212,2.661842,1.5817

In [ ]:
## Subset
rna_counts_hq_cts =  rna_counts_hq_cpm[patients_id]

## Check
print("Shape is: ", rna_counts_hq_cts.shape)
display(rna_counts_hq_cts.head())

Shape is:  (15924, 136)


,SU2CLC-CLE-NIVO10-T1,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO2-T1,SU2CLC-CLE-NIVO20-T1,SU2CLC-CLE-NIVO24-T1,SU2CLC-CLE-NIVO3-T1,SU2CLC-CLE-NIVO31-T1,SU2CLC-CLE-NIVO5-T1,SU2CLC-CLE-NIVO52-T1,SU2CLC-CLE-NIVO9-T1,SU2CLC-COL-1001-T1,SU2CLC-COL-1005-T1,SU2CLC-COL-1007-T1,SU2CLC-COL-1008-T1,SU2CLC-COL-1010-T1,SU2CLC-COL-1016-T1,SU2CLC-COL-1017-T1,SU2CLC-COL-1018-T1,SU2CLC-COL-1020-T1,SU2CLC-COL-1021-T1,SU2CLC-COL-1022-T1,SU2CLC-COL-1023-T1,SU2CLC-COL-1025-T1,SU2CLC-COL-1026-T1,SU2CLC-COL-1027-T1,SU2CLC-COL-1029-T1,SU2CLC-COL-1031-T1,SU2CLC-COL-1032-T1,SU2CLC-COL-1033-T1,SU2CLC-COL-1034-T1,SU2CLC-COL-1035-T1,SU2CLC-COL-1036-T1,SU2CLC-COL-1037-T1,SU2CLC-COL-1038-T1,SU2CLC-COL-1039-T2,SU2CLC-COL-1041-T1,SU2CLC-COL-1043-T2,SU2CLC-COL-1044-T1,SU2CLC-DFC-1001-T1,SU2CLC-DFC-1002-T1,SU2CLC-DFC-1003-T1,SU2CLC-DFC-1004-T1,SU2CLC-DFC-1007-T1,SU2CLC-DFC-1012-T1,SU2CLC-DFC-1013-T1,SU2CLC-DFC-1015-T2,SU2CLC-DFC-1016-T1,SU2CLC-DFC-1017-T2,SU2CLC-DFC-1018-T1,SU2CLC-DFC-1019-T1,SU2CLC-DFC-1020-T1,SU2CLC-DFC-1534-T1,SU2CLC-DFC-1535-T1,SU2CLC-DFC-1536-T1,SU2CLC-DFC-1537-T1,SU2CLC-DFC-1538-T1,SU2CLC-DFC-1539-T1,SU2CLC-DFC-DF0032-T1,SU2CLC-DFC-DF0033-T1,SU2CLC-DFC-DF0047-T1,SU2CLC-DFC-DF0107-T1,SU2CLC-DFC-DF0108-T1,SU2CLC-DFC-DF0109-T1,SU2CLC-DFC-DF0112-T1,SU2CLC-DFC-DF0241-T1,SU2CLC-DFC-DF0499-T1,SU2CLC-DFC-DF0510-T1,SU2CLC-DFC-DF0512-T1,SU2CLC-DFC-DF0561-T1,SU2CLC-DFC-DF0668-T1,SU2CLC-DFC-DF0790-T1,SU2CLC-DFC-DF0840-T1,SU2CLC-MDA-1441-T1,SU2CLC-MDA-1442-T1,SU2CLC-MDA-1443-T1,SU2CLC-MDA-1444-T1,SU2CLC-MDA-1561-T1,SU2CLC-MDA-1562-T1,SU2CLC-MDA-1563-T1,SU2CLC-MDA-1564-T1,SU2CLC-MDA-1627-T1,SU2CLC-MDA-1628-T1,SU2CLC-MDA-1629-T1,SU2CLC-MDA-1630-T1,SU2CLC-MDA-1631-T1,SU2CLC-MGH-1044-T1,SU2CLC-MGH-1054-T2,SU2CLC-MGH-1055-T1,SU2CLC-MGH-1135-T2,SU2CLC-MGH-1148-T1,SU2CLC-MGH-1149-T1,SU2CLC-MGH-1150-T1,SU2CLC-MGH-1161-T2,SU2CLC-MGH-1163-T1,SU2CLC-MGH-1169-T1,SU2CLC-MGH-1387-T1,SU2CLC-MGH-1389-T1,SU2CLC-MGH-1409-T1,SU2CLC-MGH-1411-T1,SU2CLC-MGH-1412-T1,SU2CLC-MGH-1413-T1,SU2CLC-MGH-1414-T1,SU2CLC-MGH-1415-T1,SU2CLC-MGH-1416-T1,SU2CLC-MGH-1417-T1,SU2CLC-MGH-1418-T1,SU2CLC-MGH-1487-T1,SU2CLC-MGH-1488-T1,SU2CLC-MGH-1489-T1,SU2CLC-MGH-1490-T1,SU2CLC-MGH-1492-T1,SU2CLC-MGH-1493-T1,SU2CLC-MGH-1495-T1,SU2CLC-MGH-1498-T1,SU2CLC-MGH-1500-T1,SU2CLC-MGH-1501-T1,SU2CLC-MGH-1503-T1,SU2CLC-MGH-1565-T1,SU2CLC-MGH-1568-T1,SU2CLC-MGH-1572-T1,SU2CLC-MGH-1574-T1,SU2CLC-MGH-1575-T1,SU2CLC-MSK-1364-T1,SU2CLC-MSK-1365-T1,SU2CLC-MSK-A2009-T1,SU2CLC-MSK-A2013-T1,SU2CLC-MSK-A2014-T1,SU2CLC-MSK-A2060-T1,SU2CLC-MSK-A2075-T1,SU2CLC-UCD-1124-T1,SU2CLC-UCD-1137-T1,SU2CLC-UCD-1142-T1,SU2CLC-UCD-1143-T1,SU2CLC-UCD-1557-T1,SU2CLC-UCD-1560-T1
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634.11,4.431076,0.453236,1.155171,1.009496,1.767499,3.307169,0.360152,0.850265,5.292916,52.776752,2.684743,4.189089,7.803944,0.928295,0.553878,1.350860,1.682919,0.306656,0.000000,7.350413,2.353301,4.706567,3.533557,0.432397,0.860571,4.574732,1.306453,0.495657,4.650064,17.301699,0.335699,7.226855,1.783137,1.114443,1.586550,0.926229,1.926702,3.643740,0.879785,7.798641,2.771499,17.313346,2.388004,5.376212,6.243532,3.387685,4.052297,0.651425,2.532179,32.106544,2.183914,1.458088,4.709117,6.184516,1.502689,5.528765,3.639040,3.898140,7.210490,3.907967,2.027742,2.882646,6.936726,5.329858,4.736300,0.930876,2.706959,0.427445,2.640967,0.092825,1.192969,0.480931,4.105426,8.984268,0.881750,4.555242,0.811472,5.054237,1.412726,2.761905,0.000000,6.881449,7.022262,2.075339,18.495830,0.557860,8.206746,8.156931,0.000000,5.334674,0.754773,6.509234,3.244957,1.480543,15.500587,1.390940,9.337178,3.920356,3.102597,0.994180,0.520856,5.577870,1.885142,0.966936,2.321603,3.670364,3.960003,0.027218,5.111210,0.543567,3.723049,1.090681,0.880358,7.600478,1.922739,0.875893,0.148757,1.208175,0.252068,2.935332,13.906862,4.539829,1.619512,13.348662,2.940581,3.946299,7.682567,1.270299,4.439976,2.985712,5.150212,2.661842,1.5817

Now, it's possible to extract the two different datasets.

- rna_counts_pr_cr: contains the RNA-seq for response group
- rna_counts_sd_pd: contains the RNA-seq for resistance group

In [ ]:
## Extract the group of patients
columns_to_select_pr_cr = rna_patients_pr_cr["Harmonized_SU2C_RNA_Tumor_Sample_ID_v2"].values.tolist()
## Extract Dataset
rna_counts_pr_cr_df = rna_counts_hq_cts[columns_to_select_pr_cr]

## Check
print("Patients for Response group (PR/CR)")
print("\n")
print("Shape is: ", rna_counts_pr_cr_df.shape)
display(rna_counts_pr_cr_df.head())

Patients for Response group (PR/CR)


Shape is:  (15924, 52)


,SU2CLC-CLE-NIVO19-T1,SU2CLC-CLE-NIVO20-T1,SU2CLC-CLE-NIVO3-T1,SU2CLC-CLE-NIVO5-T1,SU2CLC-CLE-NIVO52-T1,SU2CLC-COL-1005-T1,SU2CLC-COL-1017-T1,SU2CLC-COL-1018-T1,SU2CLC-COL-1025-T1,SU2CLC-COL-1029-T1,SU2CLC-COL-1036-T1,SU2CLC-COL-1037-T1,SU2CLC-COL-1039-T2,SU2CLC-DFC-1012-T1,SU2CLC-DFC-1020-T1,SU2CLC-DFC-1535-T1,SU2CLC-DFC-1536-T1,SU2CLC-DFC-1537-T1,SU2CLC-DFC-DF0241-T1,SU2CLC-DFC-DF0790-T1,SU2CLC-MDA-1627-T1,SU2CLC-MDA-1631-T1,SU2CLC-MGH-1044-T1,SU2CLC-MGH-1054-T2,SU2CLC-MGH-1055-T1,SU2CLC-MGH-1161-T2,SU2CLC-MGH-1163-T1,SU2CLC-MGH-1387-T1,SU2CLC-MGH-1412-T1,SU2CLC-MGH-1413-T1,SU2CLC-MGH-1414-T1,SU2CLC-MGH-1415-T1,SU2CLC-MGH-1416-T1,SU2CLC-MGH-1417-T1,SU2CLC-MGH-1418-T1,SU2CLC-MGH-1487-T1,SU2CLC-MGH-1488-T1,SU2CLC-MGH-1489-T1,SU2CLC-MGH-1490-T1,SU2CLC-MGH-1498-T1,SU2CLC-MGH-1503-T1,SU2CLC-MGH-1565-T1,SU2CLC-MGH-1574-T1,SU2CLC-MSK-1364-T1,SU2CLC-MSK-A2009-T1,SU2CLC-MSK-A2013-T1,SU2CLC-MSK-A2014-T1,SU2CLC-MSK-A2060-T1,SU2CLC-MSK-A2075-T1,SU2CLC-UCD-1124-T1,SU2CLC-UCD-1137-T1,SU2CLC-UCD-1142-T1
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634.11,1.155171,1.767499,0.360152,5.292916,52.776752,7.803944,0.306656,0.000000,0.432397,1.306453,1.783137,1.114443,0.926229,6.243532,1.458088,6.184516,1.502689,5.528765,0.930876,0.480931,6.881449,0.557860,8.206746,8.156931,0.000000,1.480543,15.500587,9.337178,0.520856,5.577870,1.885142,0.966936,2.321603,3.670364,3.960003,0.027218,5.111210,0.543567,3.723049,1.922739,1.208175,0.252068,4.539829,13.348662,3.946299,7.682567,1.270299,4.439976,2.985712,5.150212,2.661842,1.581718
ENSG00000188976.10,51.597657,105.780337,53.704539,40.825648,34.120935,50.039426,62.350654,16.635854,51.695477,52.385391,62.049038,54.283014,75.870929,36.060203,63.552540,94.995108,44.343232,26.419155,63.445043,54.996788,28.472450,78.024362,62.259033,40.735973,96.684696,51.308127,36.468435,75.996927,42.694378,34.748357,39.442981,43.129387,28.111425,39.456410,48.222419,87.460821,34.816494,20.228449,61.484533,32.252813,45.567774,28.670977,57.504498,75.225956,134.963427,53.525806,129.900408,58.173232,61.630737,90.646653,41.486701,71.838019
ENSG00000187961.13,10.847989,51.676885,10.637050,8.423136,0.000000,11.046617,8.022788,0.016838,6.720172,8.211993,8.596162,11.890314,20.121526,9.182560,19.022187,53.847938,15.945197,7.161659,24.435505,14.412416,0.206322,5.027081,11.434992,6.041971,0.000000,12.876557,11.084722,27.530236,7.765484,4.612090,5.534585,9.125461,6.386263,15.130887,6.539454,2.576647,9.921078,24.538157,10.283568,6.234335,11.306228,3.896257,15.230393,16.294573,16.705999,12.238312,9.238535,10.216719,11.963020,18.062217,8.150305,17.278765
ENSG00000187583.10,18.549132,37.776552,1.448984,2.636973,0.358140,4.803462,16.261064,0.000000,5.164743,6.430466,9.049677,4.054115,2.076030,2.969485,30.133826,9.940246,11.353648,1.082679,7.359741,4.188754,1.456391,6.016016,3.828333,3.456418,0.000000,6.433065,4.498475,29.744206,6.976309,0.847521,2.150996,7.144585,4.620955,8.333224,2.736883,8.791446,4.728738,6.639280,8.476263,5.658161,15.518513,5.631928,4.198121,28.222885,0.000000,10.658091,7.044383,8.903822,8.170360,31.747484,3.257587,17.058526
ENSG00000187642.9,0.358501,2.995761,0.879441,0.663986,0.000000,0.053820,0.638176,0.000000,0.510469,0.169669,0.103071,6.906035,0.015969,0.548213,1.089376,3.021171,0.556551,0.088744,0.639978,0.465417,1.650577,2.434299,0.666763,2.812735,0.000000,1.449264,0.322929,3.513474,0.205186,0.275937,0.193348,0.389460,0.778813,0.955044,0.520734,0.027218,0.475192,0.000000,0.524119,2.104007,2.481656,0.777811,0.260349,1.025808,0.460402,1.714709,0.148476,0.334192,1.149902,0.335566,0.551381,0.520565


In [ ]:
## Extract the group of patients
columns_to_select_sd_pd = rna_patients_sd_pd["Harmonized_SU2C_RNA_Tumor_Sample_ID_v2"].values.tolist()
## Extract Dataset
rna_counts_sd_pd_df = rna_counts_hq_cts[columns_to_select_sd_pd]

## Check
print("Patients for Response group (SD/PD)")
print("\n")
print("Shape is: ", rna_counts_sd_pd_df.shape)
display(rna_counts_sd_pd_df.head())

Patients for Response group (SD/PD)


Shape is:  (15924, 84)


,SU2CLC-CLE-NIVO10-T1,SU2CLC-CLE-NIVO18-T1,SU2CLC-CLE-NIVO2-T1,SU2CLC-CLE-NIVO24-T1,SU2CLC-CLE-NIVO31-T1,SU2CLC-CLE-NIVO9-T1,SU2CLC-COL-1001-T1,SU2CLC-COL-1007-T1,SU2CLC-COL-1008-T1,SU2CLC-COL-1010-T1,SU2CLC-COL-1016-T1,SU2CLC-COL-1020-T1,SU2CLC-COL-1021-T1,SU2CLC-COL-1022-T1,SU2CLC-COL-1023-T1,SU2CLC-COL-1026-T1,SU2CLC-COL-1027-T1,SU2CLC-COL-1031-T1,SU2CLC-COL-1032-T1,SU2CLC-COL-1033-T1,SU2CLC-COL-1034-T1,SU2CLC-COL-1035-T1,SU2CLC-COL-1038-T1,SU2CLC-COL-1041-T1,SU2CLC-COL-1043-T2,SU2CLC-COL-1044-T1,SU2CLC-DFC-1001-T1,SU2CLC-DFC-1002-T1,SU2CLC-DFC-1003-T1,SU2CLC-DFC-1004-T1,SU2CLC-DFC-1007-T1,SU2CLC-DFC-1013-T1,SU2CLC-DFC-1015-T2,SU2CLC-DFC-1016-T1,SU2CLC-DFC-1017-T2,SU2CLC-DFC-1018-T1,SU2CLC-DFC-1019-T1,SU2CLC-DFC-1534-T1,SU2CLC-DFC-1538-T1,SU2CLC-DFC-1539-T1,SU2CLC-DFC-DF0032-T1,SU2CLC-DFC-DF0033-T1,SU2CLC-DFC-DF0047-T1,SU2CLC-DFC-DF0107-T1,SU2CLC-DFC-DF0108-T1,SU2CLC-DFC-DF0109-T1,SU2CLC-DFC-DF0112-T1,SU2CLC-DFC-DF0499-T1,SU2CLC-DFC-DF0510-T1,SU2CLC-DFC-DF0512-T1,SU2CLC-DFC-DF0561-T1,SU2CLC-DFC-DF0668-T1,SU2CLC-DFC-DF0840-T1,SU2CLC-MDA-1441-T1,SU2CLC-MDA-1442-T1,SU2CLC-MDA-1443-T1,SU2CLC-MDA-1444-T1,SU2CLC-MDA-1561-T1,SU2CLC-MDA-1562-T1,SU2CLC-MDA-1563-T1,SU2CLC-MDA-1564-T1,SU2CLC-MDA-1628-T1,SU2CLC-MDA-1629-T1,SU2CLC-MDA-1630-T1,SU2CLC-MGH-1135-T2,SU2CLC-MGH-1148-T1,SU2CLC-MGH-1149-T1,SU2CLC-MGH-1150-T1,SU2CLC-MGH-1169-T1,SU2CLC-MGH-1389-T1,SU2CLC-MGH-1409-T1,SU2CLC-MGH-1411-T1,SU2CLC-MGH-1492-T1,SU2CLC-MGH-1493-T1,SU2CLC-MGH-1495-T1,SU2CLC-MGH-1500-T1,SU2CLC-MGH-1501-T1,SU2CLC-MGH-1568-T1,SU2CLC-MGH-1572-T1,SU2CLC-MGH-1575-T1,SU2CLC-MSK-1365-T1,SU2CLC-UCD-1143-T1,SU2CLC-UCD-1557-T1,SU2CLC-UCD-1560-T1
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634.11,4.431076,0.453236,1.009496,3.307169,0.850265,2.684743,4.189089,0.928295,0.553878,1.350860,1.682919,7.350413,2.353301,4.706567,3.533557,0.860571,4.574732,0.495657,4.650064,17.301699,0.335699,7.226855,1.586550,1.926702,3.643740,0.879785,7.798641,2.771499,17.313346,2.388004,5.376212,3.387685,4.052297,0.651425,2.532179,32.106544,2.183914,4.709117,3.639040,3.898140,7.210490,3.907967,2.027742,2.882646,6.936726,5.329858,4.736300,2.706959,0.427445,2.640967,0.092825,1.192969,4.105426,8.984268,0.881750,4.555242,0.811472,5.054237,1.412726,2.761905,0.000000,7.022262,2.075339,18.495830,5.334674,0.754773,6.509234,3.244957,1.390940,3.920356,3.102597,0.994180,1.090681,0.880358,7.600478,0.875893,0.148757,2.935332,13.906862,1.619512,2.940581,2.516983,2.945791,0.872072
ENSG00000188976.10,25.584913,49.855911,38.227745,88.426113,0.000000,36.342253,48.361057,24.739552,67.890740,69.577832,102.786623,32.127086,38.556317,38.393164,52.607910,51.051552,30.327320,17.372767,61.323629,46.605695,74.629457,39.108726,42.290065,46.041244,33.718329,56.423081,25.937886,34.855127,77.221622,53.431587,39.530971,71.902390,46.851839,41.066923,78.621074,79.724301,29.586835,32.655485,27.654637,78.085247,52.399899,41.987386,47.651945,43.077746,58.091516,56.337918,70.777123,166.411429,54.452189,39.793555,44.494215,144.681455,49.631097,38.931827,31.976931,64.294865,59.663512,40.874157,41.788435,107.150353,20.389659,49.725347,65.614057,52.583473,49.978231,41.690086,49.419003,80.743658,73.957881,81.839175,28.228684,30.819585,47.141646,37.030753,27.971127,42.117948,35.372220,57.891820,67.241338,87.911023,18.007987,31.447360,94.265307,72.963388
ENSG00000187961.13,3.186733,9.291329,9.640137,20.620109,17.643009,8.591178,9.134985,0.973212,15.019859,8.618144,7.830249,2.890612,6.912822,6.355060,13.394351,8.076813,8.498748,10.619445,12.314600,10.590737,9.602161,16.898005,5.342280,12.367346,2.783158,13.457960,6.120453,14.092369,23.553181,8.818202,11.815017,13.018858,7.683428,7.247104,7.571833,30.271884,6.770133,12.851966,5.686000,8.469780,16.760262,12.700893,17.134423,21.474095,22.865506,16.899907,24.693691,29.820928,12.758157,12.443879,12.252927,25.535575,16.501266,9.612538,4.075844,6.733169,8.905910,9.668209,9.521773,19.492399,0.000000,7.535838,15.861517,8.838817,13.249942,

At this point the two datasets will be normalized in order to be used for network analysis.

In [ ]:
## Save dataset for Response group
with open(save_path_data + "rna_counts_pr_cr_normalized.csv", "w") as output:
    output.write(str(rna_counts_pr_cr_df.to_csv(index = True)))

In [ ]:
## Save dataset for Resistance group
with open(save_path_data + "rna_counts_sd_pd_normalized.csv", "w") as output:
    output.write(str(rna_counts_sd_pd_df.to_csv(index = True)))

In [ ]:
## Save the dataset that will be the input of Limma
with open(save_path_data + "rna_counts_hq.csv", "w") as output:
    output.write(str(rna_counts_hq_cts.to_csv()))